## Imports

In [ ]:
import torch
import os
import json
import matplotlib.pyplot as plt

In [ ]:
import matplotlib
matplotlib.rcParams['figure.facecolor'] = 'white'

In [ ]:
%run ../iu_xray.py

In [ ]:
REPORTS_DIR = os.path.join(DATASET_DIR, 'reports')

## Apply chexpert labeler to reports

In [ ]:
%run -n ../../eval_report_generation_chexpert_labeler.py

In [ ]:
fname = os.path.join(REPORTS_DIR, 'reports.clean.json')
with open(fname, 'r') as f:
    clean_reports = list(json.load(f).values())
len(clean_reports)

In [ ]:
reports_by_filename = {
    r['filename']: r['clean_text']
    for r in clean_reports
}
len(reports_by_filename)

In [ ]:
df_reports = pd.DataFrame(reports_by_filename, columns=['reports'])
print(len(df_reports))
df_reports.head()

In [ ]:
%%time

labels = _apply_labeler_to_column(df_reports, 'reports',
                                  fill_empty=-2, fill_uncertain=-1)
labels.shape

In [ ]:
df_reports = _concat_df_matrix(df_reports, labels)
df_reports.head()

## Apply MIRQI labeler to reports

In [ ]:
from collections import defaultdict

In [ ]:
%run -n ../../eval_report_generation_mirqi.py

In [ ]:
path = os.path.join(REPORTS_DIR, 'reports_with_chexpert_labels.csv')
df = pd.read_csv(path, index_col=0)
df = df[['Reports', 'filename']]
df.head()

In [ ]:
%%time

mirqi_df = _apply_mirqi_to_df(df, gt_col_name='Reports', gen_col_name='Reports')
mirqi_df.head()

In [ ]:
valoration_to_int = {
    'POSITIVE': 1,
    'NEGATIVE': 0,
    'UNCERTAIN': -1,
}

WRONG_LEN_ATTRIBUTES = defaultdict(list)

def expand_attributes(row):
    attributes = row['attributes-gt']
    attributes = [s.strip('()') for s in attributes.split(') (')]
    attributes = [s.split('|') for s in attributes]
    
    for tup in attributes:
        if len(tup) != 4:
            WRONG_LEN_ATTRIBUTES['len-not-4'].append(tup)
            continue
        text, label, value, additional = tup
        row[label] = valoration_to_int[value]
        
    return row

In [ ]:
mirqi_df = mirqi_df.apply(expand_attributes, axis=1)
mirqi_df.head()

In [ ]:
base_cols = ['filename', 'Reports', 'attributes-gt', 'attributes-gen', 'MIRQI-r', 'MIRQI-p', 'MIRQI-f']
columns = base_cols + [c for c in mirqi_df if c not in base_cols]
mirqi_df = mirqi_df[columns]
mirqi_df.head()

In [ ]:
mirqi_df.replace(np.nan, -2, inplace=True)
mirqi_df.head()

In [ ]:
path = os.path.join(REPORTS_DIR, 'reports_with_mirqi_labels.csv')
mirqi_df.to_csv(path)

## Split sentences

In [ ]:
%run ../../utils/nlp.py

In [ ]:
from collections import Counter, defaultdict

In [ ]:
reports_fname = os.path.join(REPORTS_DIR, 'reports.clean.v3.json')
with open(reports_fname, 'r') as f:
    reports_as_dict = json.load(f)
    reports = list(reports_as_dict.values())
len(reports_as_dict), len(reports)

### Count appearances

In [ ]:
sentence_counter = get_sentences_appearances(r['clean_text'] for r in reports)
len(sentence_counter)

In [ ]:
sentence_counter

#### Check not-so-common sentences

In [ ]:
l = list(sentence_counter.items())
l = sorted(l, key=lambda x: x[1], reverse=False)
l[:20]

#### Check short sentences

In [ ]:
sorted(sentence_counter.items(), key=lambda x: len(x[0]))

In [ ]:
target_sentence = 'hand .'
appears_in = [
    report
    for report in reports
    if target_sentence in report['clean_text']
]
len(appears_in), appears_in

## Label sentences with chexpert labels

In [ ]:
%run -n ../../metrics/report_generation/chexpert.py
# %run -n ../../eval_report_generation_chexpert_labeler.py

In [ ]:
columns = ['sentences', 'appearances']
df_sentences = pd.DataFrame(sentence_counter.items(), columns=columns)
print(len(df_sentences))
df_sentences.head()

In [ ]:
%%time

labels = apply_labeler_to_column(df_sentences, 'sentences',
                                 fill_empty=-2, fill_uncertain=-1,
                                 caller_id='iu-notebook')
labels.shape

In [ ]:
df_sentences = _concat_df_matrix(df_sentences, labels)
print(len(df_sentences))
df_sentences.head()

In [ ]:
fpath = os.path.join(REPORTS_DIR, 'sentences_with_chexpert_labels.csv')

In [ ]:
df_sentences.to_csv(fpath, index=False)

## Load sentences for later use

In [ ]:
import pandas as pd

In [ ]:
fpath = os.path.join(REPORTS_DIR, 'sentences_with_chexpert_labels.csv')

In [ ]:
df_sentences = pd.read_csv(fpath)
df_sentences.head()

In [ ]:
len(df_sentences)

## Count sentences' groups

### Count normal vs abnormal

In [ ]:
snt_normal = df_sentences[df_sentences['No Finding'] == 1]
snt_abnormal = df_sentences[df_sentences['No Finding'] == 0]
len(snt_normal), len(snt_abnormal)

In [ ]:
snt_normal['appearances'].sum()

In [ ]:
snt_abnormal['appearances'].sum()

### Number of abnormal sentences per report

In [ ]:
is_sentence_normal = dict()
for index, row in df_sentences.iterrows():
    sentence = row['sentences']
    is_normal = row['No Finding']
    is_sentence_normal[sentence] = is_normal
len(is_sentence_normal)

In [ ]:
res = []
for report in reports:
    number_of_abnormal = sum(
        1 - is_sentence_normal[sentence]
        for sentence in split_sentences(report['clean_text'])
    )
    res.append(number_of_abnormal)
len(res)

In [ ]:
plt.hist(res)

### Top-K most common sentences

In [ ]:
cols = ['sentences', 'appearances']
df = df_sentences[cols].sort_values('appearances', ascending=False).head(5)
df

### Plot sentences appearances distribution

In [ ]:
len(df_sentences)

In [ ]:
df_sentences.sort_values('appearances', ascending=False).head()

In [ ]:
appearances = list(df_sentences['appearances'])

In [ ]:
plt.hist(appearances, bins=30)
plt.yscale('log')

plt.title('Sentence appearances distribution')

plt.ylabel('Number of sentences')
plt.xlabel('Number of appearances')

In [ ]:
plt.bar(list(range(len(values))), values)

## Collect synonyms

In [ ]:
import json
from collections import Counter, defaultdict

In [ ]:
%run ../vocab/__init__.py

In [ ]:
SYNONYMS = load_synonyms('iu_xray')
len(SYNONYMS)

In [ ]:
SYNONYMS = {}

In [ ]:
FOR_LATER = set()

In [ ]:
SEEN_SENTENCES = set()
for representative, syns in SYNONYMS.items():
    SEEN_SENTENCES.add(representative)
    for s in syns:
        SEEN_SENTENCES.add(s)
len(SEEN_SENTENCES)

In [ ]:
def is_number(s):
    try:
        s = int(s)
        return True
    except:
        return False

In [ ]:
def process_sentences(sentences, skip_later=True):
    index_to_repr = {
        index: representative
        for index, representative in enumerate(SYNONYMS.keys())
    }
    
    def _print_reprs():
        print('-'*20)
        for index, representative in index_to_repr.items():
            print(f'{index} - {representative}')
    
    def _add_new(sentence):
        index_to_repr[len(SYNONYMS)] = sentence
        SYNONYMS[sentence] = []
        SEEN_SENTENCES.add(sentence)

    def _add_as_syn(sentence, option):
        option = int(option)
            
        if option not in index_to_repr:
            print(f'No synonym found for option={option}')
            raise
        representative = index_to_repr[option]

        if representative not in SYNONYMS:
            print(f'representative {representative} not in SYNS')
            # Internal error!
            raise

        SYNONYMS[representative].append(sentence)
        SEEN_SENTENCES.add(sentence)
        
    _print_reprs()
    
    sentence_idx = 0
    while sentence_idx < len(sentences):
        sentence = sentences[sentence_idx]
        sentence = clean_sentence(sentence)
        
        if sentence in SEEN_SENTENCES or (not skip_later and sentence in FOR_LATER):
            sentence_idx += 1
            continue
            
        option = input(f'"{sentence}" --> ')
        
        if is_number(option):
            _add_as_syn(sentence, option)
            sentence_idx += 1
        elif option == 'l': # later
            FOR_LATER.add(sentence)
            sentence_idx += 1
        elif ',' in option: # split and allocate
            added_new = False
            for suboption in option.split(','):
                suboption = suboption.strip()
                if suboption == 'n':
                    new_sentence = input('\t\tInput new sentence: ')
                    _add_new(new_sentence)
                    added_new = True
                else:
                    _add_as_syn(sentence, suboption)

            if added_new:
                _print_reprs()
                
            sentence_idx += 1
        elif option == 'n': # new
            _add_new(sentence)
            _print_reprs()
            sentence_idx += 1
        elif option == 'b':
            print('Breaking')
            break
        else:
            print(f'Option not recognized: {option}')

In [ ]:
some_sentences = df_sentences.groupby('Fracture')['sentences'].apply(list)
some_sentences = sorted(some_sentences[0], key=lambda x: len(x))
process_sentences(some_sentences)

In [ ]:
save_synonyms('iu_xray', SYNONYMS)

In [ ]:
o

## Inspect Max amounts

### Max amount of words in a sentence

In [ ]:
sorted([(len(s.split()), s) for s in sentence_counter], reverse=True)

 ### Max amount of words

In [ ]:
max(len(report['clean_text'].split()) for report in reports)

### Max amount of sentences

In [ ]:
max(len(split_sentences(report['clean_text'])) for report in reports)

## Sentences + organs

In [ ]:
%run ../common/sentences2organs/compute.py
%run ../common/constants.py

In [ ]:
sentences = list(df_sentences['sentences'])
len(sentences)

In [ ]:
df_organs, errors = save_sentences_with_organs(DATASET_DIR, sentences)
len(df_organs), len(errors['all-empty'])

###  Review empty ones

In [ ]:
len(errors['all-empty'])

In [ ]:
sample = df_sentences.loc[df_sentences['sentences'].isin(set(errors['all-empty']))]
len(sample)

In [ ]:
sample['appearances'].sum(), df_sentences['appearances'].sum()

In [ ]:
sample.sort_values('appearances', ascending=False).head(40)

## Compute sentence embeddings

### RadGlove average as embeddings

#### Load sentences + organs

In [ ]:
fpath = os.path.join(REPORTS_DIR, 'sentences_with_chexpert_labels.csv')
sentences_df = pd.read_csv(fpath)
sentences_df.head()

In [ ]:
sentences = list(sentences_df['sentences'])
sentences[:10]

#### Compute RadGlove average embeddings

In [ ]:
%run ../../models/report_generation/word_embedding.py

In [ ]:
radglove = RadGlove()

In [ ]:
embeddings = []

for sentence in sentences:
    words_embeddings = []
    for word in sentence.split():
        if word == '.':
            continue
        if word not in radglove:
            continue

        words_embeddings.append(radglove[word])

    if len(words_embeddings) == 0:
        words_embeddings = [radglove['UNKNOWN']]

    words_embeddings = torch.stack(words_embeddings, dim=0)
    
    # Grab mean
    sentence_embedding = words_embeddings.mean(dim=0)

    embeddings.append(sentence_embedding)
    
embeddings = torch.stack(embeddings, dim=0)
embeddings = embeddings.numpy()
embeddings.shape

In [ ]:
columns = [f'emb{i}' for i in range(embeddings.shape[1])]
embeddings_df = pd.DataFrame(embeddings, columns=columns)
embeddings_df['sentence'] = sentences
embeddings_df.head()

In [ ]:
fpath = os.path.join(REPORTS_DIR, 'sentences-embeddings', 'radglove-average.csv')
embeddings_df.to_csv(fpath, index=False)

#### Show in TB

In [ ]:
%run ../../tensorboard/__init__.py
%run ../../utils/files.py
%run ../common/sentences2organs/compute.py

In [ ]:
onehot_organs, warnings = find_organs_for_sentences(sentences)
df = pd.DataFrame([
    (sentence, get_main_organ(onehot, sentence), *onehot)
    for onehot, sentence in zip(onehot_organs, sentences)
], columns=['sentence', 'organ', *JSRT_ORGANS])
df = df.merge(embeddings_df, on='sentence')
df.head()

In [ ]:
subsample = df.sample(1000)
len(subsample)

In [ ]:
header = ['sentence', 'organ', *JSRT_ORGANS]
emb_columns = [c for c in subsample.columns if 'emb' in c]
metadata = subsample[header].to_numpy()
metadata = [tuple(map(str, x)) for x in metadata]
embeddings = subsample[emb_columns].to_numpy()
assert len(metadata) == len(embeddings)
tag = f'radglove-average-words-{len(embeddings)}'
len(metadata), tag

In [ ]:
run_id = RunId('gt-sentence-embeddings', debug=False, task='rg')
get_tb_large_log_folder(run_id)

In [ ]:
writer = SummaryWriter(get_tb_large_log_folder(run_id))

In [ ]:
writer.add_embedding(
    embeddings,
    metadata=metadata,
    metadata_header=header,
    tag=tag,
)

In [ ]:
writer.close()